# Exploratory Data Analysis

## Introduction

After cleaning the data we are going to take a look a it. And since we want to know how the information changes across time, we will be looking at tweets from different weeks.

1. **Most common words:** Find them and create word clouds. See if anything needs to be removed.
2. **Size of vocabulary:** Look at the number of unique words used
3. **Engagement metrics across time:** A much insightfull look into the stats obtained during data cleaning.


In [1]:
import json
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio
import re
import spacy

from dash import Dash, dcc, html, Input, Output
from dotenv import load_dotenv
from itertools import product
from jupyter_dash import JupyterDash

In [2]:
load_dotenv()

BASE_DIR = os.environ.get("BASE_DIR")
BEARER_TOKEN = os.environ.get("BEARER_TOKEN")

In [3]:
pd.set_option("display.max_colwidth", 300)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)
pd.set_option("display.precision", 2)
pd.set_option("display.float_format",  "{:,.2f}".format)

pio.templates.default = "plotly_white"
pio.kaleido.scope.default_scale = 2

gruvbox_colors = ["#fabd2f", "#b8bb26", "#458588", "#fe8019", "#b16286", "#fb4943", "#689d6a", "#d79921", "#98971a", "#83a598", "#d65d0e", "#d3869b", "#cc241d", "#8ec07c", "#b57614", "#79740e", "#076678", "#af3a03", "#8f3f71", "#9d0006", "#4d7b58", "#fbf1c7", "#928374", "#282828"]

In [4]:
TIME_STAMPS = [(2022, 35), (2022, 40), (2022, 45), (2022, 50), (2023, 3)]

### Data Loading

There are three documents that I want to load. The corpus frame, document term matrix and clean data.

In [5]:
corpus = pd.read_feather(f"{BASE_DIR}/data/processed/corpus-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")
dtm = pd.read_feather(f"{BASE_DIR}/data/processed/dtm-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")
data_dtm = pd.read_feather(f"{BASE_DIR}/data/processed/data-dtm-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")
stats_data = pd.read_feather(f"{BASE_DIR}/data/processed/stats_data-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")
top30_df = pd.read_feather(f"{BASE_DIR}/data/processed/top30_df-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")

In [6]:
dtm.set_index("index", inplace=True)
data_dtm.set_index("index", inplace=True)
corpus.set_index("index", inplace=True)
stats_data.set_index("index", inplace=True)

In [7]:
dtm.head()

word,aa,aaaaatención,aaaatención,aactores,aafp,aaguinaga,aahh,aap,aar,aaron,abad,abajo,abancay,abanderados,abandona,abandonada,abandonadas,abandonado,abandonados,abandonan,abandonar,abandonaron,abandonará,abandonen,abandono,...,ünsal,​chocolate,​mantequilla,⃣,→,↓,─,⦿,𝗔𝘂𝗱𝗶𝘁𝗼𝗿𝗶𝗼,𝗖𝗮𝘁𝗮́𝗹𝗼𝗴𝗼,𝗗𝗲𝗹,𝗘𝗱𝗶𝘁𝗼𝗿𝗮,𝗣𝗮𝘁𝗿𝗶𝗰𝗶𝗮,𝗣𝗲𝗿𝘂́,𝗦𝗮́𝗯𝗮𝗱𝗼,𝗨́𝗻𝗲𝘁𝗲,𝗩𝗮𝗹𝗹𝗲,𝗱𝗲,𝗱𝗲𝗹,𝗲𝗱𝗶𝘁𝗼𝗿𝗶𝗮𝗹,𝗵𝗿𝘀,𝗹𝗮,𝗻𝗼𝘃𝗶𝗲𝗺𝗯𝗿𝗲,𝗽𝗿𝗲𝘀𝗲𝗻𝘁𝗮𝗰𝗶𝗼́𝗻,󠁧󠁢󠁥󠁮󠁧󠁿
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1558966707611385861,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1558966968039997441,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1558967193043361792,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1558967616777109510,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1558968396674473985,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [8]:
corpus.head()

,id,created_at,newspaper,corpus
index,,,,
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años
2,1564032331706470401,2022-08-28 23:29:00+00:00,elcomercio_peru,amlo afirma que familias ya aceptaron plan de rescate de mineros
3,1564028601053347843,2022-08-28 23:14:11+00:00,elcomercio_peru,zelensky los ocupantes rusos sentirán las consecuencias de futuras acciones
5,1564023766937731073,2022-08-28 22:54:58+00:00,elcomercio_peru,autoridades confirman transmisión comunitaria de viruela del mono en panamá
7,1564017585561141248,2022-08-28 22:30:25+00:00,elcomercio_peru,las imágenes de los enfrentamientos entre seguidores de cristina kirchner la policía en argentina


In [9]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34924 entries, 0 to 23
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          34924 non-null  object             
 1   created_at  34924 non-null  datetime64[ns, UTC]
 2   newspaper   34924 non-null  object             
 3   corpus      34924 non-null  object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 1.3+ MB


In [10]:
stats_data.head()

,created_at,possibly_sensitive,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,edit_history_tweet_ids,impression_count
index,,,,,,,,,,,
0,2022-08-28 23:57:24+00:00,False,1564039479391838209,0,0,6,1,None,elcomercio_peru,None,NaN
2,2022-08-28 23:29:00+00:00,False,1564032331706470401,0,0,2,0,None,elcomercio_peru,None,NaN
3,2022-08-28 23:14:11+00:00,False,1564028601053347843,6,7,18,1,None,elcomercio_peru,None,NaN
5,2022-08-28 22:54:58+00:00,False,1564023766937731073,1,0,1,1,None,elcomercio_peru,None,NaN
7,2022-08-28 22:30:25+00:00,False,1564017585561141248,3,0,8,0,None,elcomercio_peru,None,NaN


## Most Common Words

In [11]:
newspapers = corpus["newspaper"].unique()

In [12]:
year_weeks = corpus["created_at"].dt.isocalendar()[["year", "week"]]
year_weeks.drop_duplicates(inplace=True)
year_weeks = year_weeks.to_numpy()

In [13]:
dtm_newspaper = pd.DataFrame(index=dtm.columns)

In [14]:
corpus["year"] = corpus["created_at"].dt.isocalendar().year
corpus["week"] = corpus["created_at"].dt.isocalendar().week

In [15]:
dtm_newspaper = pd.DataFrame(index=dtm.columns)

for year_week, newspaper in product(year_weeks, newspapers):
    data_ids = corpus.loc[(corpus["newspaper"] == newspaper) & (corpus["year"] == year_week[0]) & (corpus["week"] == year_week[1]) , ["id"]]
    filtered_data = dtm.filter(items=data_ids["id"], axis=0)
    dtm_newspaper[f"{newspaper}-{year_week[0]}_{year_week[1]}"] = filtered_data.sum(axis=0)

In [16]:
top30_dict = {}

for newspaper in dtm_newspaper.columns:
    top = dtm_newspaper[newspaper].sort_values(ascending=False).head(30)
    top30_dict[newspaper] = list(zip(top.index, top.values))

In [17]:
top30_dict

{'elcomercio_peru-2022_34': [('agosto', 40.0),
  ('perú', 32.0),
  ('lima', 23.0),
  ('millones', 21.0),
  ('covid', 19.0),
  ('perucheck', 19.0),
  ('años', 17.0),
  ('mundo', 14.0),
  ('eeuu', 14.0),
  ('ucrania', 14.0),
  ('méxico', 13.0),
  ('colombia', 13.0),
  ('mono', 12.0),
  ('rusia', 12.0),
  ('viruela', 12.0),
  ('muerte', 12.0),
  ('venezuela', 12.0),
  ('tipo', 11.0),
  ('cambio', 11.0),
  ('dólar', 11.0),
  ('us', 11.0),
  ('reporta', 10.0),
  ('mujer', 10.0),
  ('contagios', 9.0),
  ('unidos', 9.0),
  ('policía', 9.0),
  ('precio', 9.0),
  ('país', 9.0),
  ('bono', 8.0),
  ('pasó', 8.0)],
 'larepublica_pe-2022_34': [('lrdeportes', 214.0),
  ('politicalr', 199.0),
  ('video', 141.0),
  ('perú', 69.0),
  ('envivo', 57.0),
  ('paredes', 56.0),
  ('castillo', 52.0),
  ('lima', 51.0),
  ('agosto', 50.0),
  ('años', 45.0),
  ('verificadorlr', 43.0),
  ('yenifer', 39.0),
  ('pedro', 35.0),
  ('partido', 35.0),
  ('prisión', 33.0),
  ('alianza', 29.0),
  ('pierdas', 28.0),
  ('l

In [18]:
top30_df = pd.DataFrame.from_records(top30_dict)

In [19]:
top30_df = top30_df.melt(value_vars=top30_df.columns, var_name="newspaper_date", value_name="word_count")

top30_df[["newspaper", "year_week"]] = top30_df["newspaper_date"].str.split(r"-", expand=True)
top30_df[["year", "week"]] = top30_df["year_week"].str.split(r"_", expand=True)
top30_df[["word", "count"]] = pd.DataFrame(top30_df["word_count"].to_list(), index=top30_df.index)

top30_df.drop(["word_count", "newspaper_date", "year_week"], axis=1, inplace=True)

top30_df["year"] = pd.to_numeric(top30_df["year"])
top30_df["week"] = pd.to_numeric(top30_df["week"])

In [20]:
top30_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   newspaper  2160 non-null   object 
 1   year       2160 non-null   int64  
 2   week       2160 non-null   int64  
 3   word       2160 non-null   object 
 4   count      2160 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 84.5+ KB


In [21]:
top30_df.head()

,newspaper,year,week,word,count
0,DiarioElPeruano,2022,33,nacional,29.00
1,DiarioElPeruano,2022,33,presidente,27.00
2,DiarioElPeruano,2022,33,país,24.00
3,DiarioElPeruano,2022,33,pedro,22.00
4,DiarioElPeruano,2022,33,castillo,22.00


In [22]:
top30_df.to_feather(f"{BASE_DIR}/data/processed/top30_df-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.feather")

In [23]:
top30_df["hot_topics"] = top30_df["word"].map({
    "castillo": "castillo",
    "pedro": "castillo",
    "dina": "boluarte",
    "boluarte": "boluarte",
    "perú": "país",
    "país": "país",
    "congreso": "congreso",
    "covid": "covid",
    "protestas": "protestas",
    "manifestaciones": "protestas"
})
top30_df["hot_topics"].fillna("", inplace=True)

In [24]:
with open(f"{BASE_DIR}/data/processed/top_30-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.json", "w") as file:
    json.dump(top30_dict, file)

In [25]:
fig = px.bar(
    top30_df,
    x="word",
    y="count",
    facet_row="newspaper",
    facet_col="week",
    color="hot_topics",
    color_discrete_sequence=gruvbox_colors,
    title="Top 30 words per newspaper per week",
    height=3200,
    width=3200
    )

fig.for_each_annotation(lambda a: a.update(text=f"{a.text.split('=')[-1]}"))
fig.update_xaxes(matches=None, showticklabels=True, categoryorder='total descending')
fig.update_yaxes(matches=None, showticklabels=True)

fig.write_html(f"{BASE_DIR}/reports/top30_bar-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.html")

fig.show()

## Number of words

In [26]:
unique_list = []

# Identify the non-zero items in the document-term matrix
for newspaper in dtm_newspaper.columns:
    uniques = dtm_newspaper[newspaper].to_numpy().nonzero()[0].size
    unique_list.append(uniques)

# Create a new datafra,e that contains this unique word count
data_words = pd.DataFrame(list(zip(dtm_newspaper.columns, unique_list)), columns=['newspaper', 'unique_words'])
data_words.set_index('newspaper', inplace=True)
data_words.sort_values(by='unique_words', ascending=False)
data_words.reset_index(inplace=True)

In [27]:
data_words[["newspaper", "year_week"]] = data_words["newspaper"].str.split(r"-", expand=True)
data_words[["year", "week"]] = data_words["year_week"].str.split(r"_", expand=True)

data_words.drop(["year_week"], axis=1, inplace=True)

data_words["year"] = pd.to_numeric(data_words["year"])
data_words["week"] = pd.to_numeric(data_words["week"])

In [28]:
data_words.head()

,newspaper,unique_words,year,week
0,elcomercio_peru,2128,2022,34
1,larepublica_pe,4841,2022,34
2,peru21noticias,4087,2022,34
3,tromepe,3290,2022,34
4,Gestionpe,2187,2022,34


In [29]:
data_words.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   newspaper     72 non-null     object
 1   unique_words  72 non-null     int64 
 2   year          72 non-null     int64 
 3   week          72 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.4+ KB


Since the number of unique words might be linked to the number of tweets, I will add a column with the number of tweets for each newspaper.

In [30]:
tweet_number = pd.DataFrame(corpus.groupby(by=["newspaper", "year", "week"]).count()["id"])
tweet_number.rename(columns={'id':'tweet_number'}, inplace=True)

In [31]:
tweet_number.reset_index(inplace=True)

In [32]:
tweet_number.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   newspaper     70 non-null     object
 1   year          70 non-null     UInt32
 2   week          70 non-null     UInt32
 3   tweet_number  70 non-null     int64 
dtypes: UInt32(2), int64(1), object(1)
memory usage: 1.9+ KB


In [33]:
data_words = data_words.merge(tweet_number)

In [34]:
data_words.head()

,newspaper,unique_words,year,week,tweet_number
0,elcomercio_peru,2128,2022,34,457
1,larepublica_pe,4841,2022,34,827
2,peru21noticias,4087,2022,34,961
3,tromepe,3290,2022,34,807
4,Gestionpe,2187,2022,34,848


In [35]:
data_words.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 69
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   newspaper     70 non-null     object
 1   unique_words  70 non-null     int64 
 2   year          70 non-null     int64 
 3   week          70 non-null     int64 
 4   tweet_number  70 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 3.3+ KB


In [36]:
data_words["word_tweet_ratio"] = data_words["unique_words"]/data_words["tweet_number"]
data_words.sort_values(by='word_tweet_ratio', ascending=False)

,newspaper,unique_words,year,week,tweet_number,word_tweet_ratio
11,ensustrece,342,2022,34,24,14.25
57,ensustrece,261,2022,49,22,11.86
69,ensustrece,186,2023,2,16,11.62
33,ensustrece,215,2022,39,19,11.32
45,ensustrece,59,2022,44,6,9.83
42,DiarioElPeruano,1729,2022,44,186,9.30
8,DiarioElPeruano,1745,2022,34,200,8.72
20,DiarioElPeruano,1883,2022,33,229,8.22
10,elbuho_pe,474,2022,34,58,8.17
54,DiarioElPeruano,1623,2022,49,201,8.07


In [37]:
data_words.to_csv(f'{BASE_DIR}/reports/tables/words_tweets-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.csv')

In [38]:
fig = px.scatter(
    data_words,
    "unique_words",
    "tweet_number",
    facet_col="week",
    color="newspaper",
    color_discrete_sequence=gruvbox_colors,
    title="Unique words per newspaper",
    width=2400,
    height=600
)

fig.show()

In [39]:
data_words = pd.read_csv(f'{BASE_DIR}/reports/tables/words_tweets-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.csv', index_col=0)

## Engagement

Now I will be looking into engagement metrics as a whole per newspaper and how it changes, as well as the relationship with the most used words as well as the ammount of vocabulary expressed in the most used words.

### Raw stats

In [40]:
stats_data.head()

,created_at,possibly_sensitive,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,edit_history_tweet_ids,impression_count
index,,,,,,,,,,,
0,2022-08-28 23:57:24+00:00,False,1564039479391838209,0,0,6,1,None,elcomercio_peru,None,NaN
2,2022-08-28 23:29:00+00:00,False,1564032331706470401,0,0,2,0,None,elcomercio_peru,None,NaN
3,2022-08-28 23:14:11+00:00,False,1564028601053347843,6,7,18,1,None,elcomercio_peru,None,NaN
5,2022-08-28 22:54:58+00:00,False,1564023766937731073,1,0,1,1,None,elcomercio_peru,None,NaN
7,2022-08-28 22:30:25+00:00,False,1564017585561141248,3,0,8,0,None,elcomercio_peru,None,NaN


In [41]:
stats_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34924 entries, 0 to 23
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   created_at              34924 non-null  datetime64[ns, UTC]
 1   possibly_sensitive      34924 non-null  bool               
 2   id                      34924 non-null  object             
 3   retweet_count           34924 non-null  int64              
 4   reply_count             34924 non-null  int64              
 5   like_count              34924 non-null  int64              
 6   quote_count             34924 non-null  int64              
 7   referenced_tweets       3593 non-null   object             
 8   newspaper               34924 non-null  object             
 9   edit_history_tweet_ids  21727 non-null  object             
 10  impression_count        4499 non-null   float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(

In [42]:
stats_data["year"] = stats_data["created_at"].dt.isocalendar().year
stats_data["week"] = stats_data["created_at"].dt.isocalendar().week

In [43]:
stats_summary = stats_data[["newspaper", "retweet_count", "reply_count", "like_count", "quote_count", "year", "week"]].groupby(by=["newspaper", "year", "week"]).agg(func=["count", "min", "mean", "std", "max", "sum"])

In [44]:
stats_summary[("retweet_count", "ratio")] = stats_summary[("retweet_count", "sum")] / stats_summary[("retweet_count", "count")]
stats_summary[("reply_count", "ratio")] = stats_summary[("reply_count", "sum")] / stats_summary[("reply_count", "count")]
stats_summary[("like_count", "ratio")] = stats_summary[("like_count", "sum")] / stats_summary[("like_count", "count")]
stats_summary[("quote_count", "ratio")] = stats_summary[("quote_count", "sum")] / stats_summary[("quote_count", "count")]

In [45]:
stats_summary = stats_summary.stack()
stats_summary = stats_summary.melt(var_name="metric", ignore_index=False)

In [46]:
stats_summary = stats_summary.reset_index()
stats_summary.rename({"level_3": "stat"}, axis=1, inplace=True)

In [47]:
stats_summary["year_week"] = stats_summary["year"].astype("str") + "w" + stats_summary["week"].astype("str")

In [48]:
stats_summary.head()

,newspaper,year,week,stat,metric,value,year_week
0,DiarioElPeruano,2022,33,count,like_count,229.00,2022w33
1,DiarioElPeruano,2022,33,max,like_count,80.00,2022w33
2,DiarioElPeruano,2022,33,mean,like_count,12.46,2022w33
3,DiarioElPeruano,2022,33,min,like_count,0.00,2022w33
4,DiarioElPeruano,2022,33,ratio,like_count,12.46,2022w33


In [49]:
stats_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1960 entries, 0 to 1959
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   newspaper  1960 non-null   object 
 1   year       1960 non-null   UInt32 
 2   week       1960 non-null   UInt32 
 3   stat       1960 non-null   object 
 4   metric     1960 non-null   object 
 5   value      1960 non-null   float64
 6   year_week  1960 non-null   object 
dtypes: UInt32(2), float64(1), object(4)
memory usage: 95.8+ KB


In [50]:
fig = px.line(
    stats_summary,
    x="year_week",
    y="value",
    color="newspaper",
    facet_row="metric",
    facet_row_spacing=0.08,
    facet_col="stat",
    color_discrete_sequence=gruvbox_colors,
    title="Raw engagement stats per newspaper",
    width=2100,
    height=1200
)

fig.for_each_annotation(lambda a: a.update(text=f"{a.text.split('=')[-1]}"))
fig.update_xaxes(showticklabels=True, tickangle = -45)
fig.update_yaxes(matches=None, showticklabels=True)

fig.show()

In [51]:
stats_summary.to_csv(f'{BASE_DIR}/reports/tables/raw_stats_summary-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.csv')

### Stats for most used words

With this bit I want to find out which are the words that drive the most engagement for the newspaper durin the time period selected. For that I'll be building a small dashboard application using [Plotly Dash](https://dash.plotly.com)

In [52]:
top30_df.head()

,newspaper,year,week,word,count,hot_topics
0,DiarioElPeruano,2022,33,nacional,29.00,
1,DiarioElPeruano,2022,33,presidente,27.00,
2,DiarioElPeruano,2022,33,país,24.00,país
3,DiarioElPeruano,2022,33,pedro,22.00,castillo
4,DiarioElPeruano,2022,33,castillo,22.00,castillo


In [53]:
data_dtm.head()

,id,created_at,newspaper,corpus,word
index,,,,,
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años,venezuela
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años,colombia
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años,retoman
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años,relaciones
0,1564039479391838209,2022-08-28 23:57:24+00:00,elcomercio_peru,venezuela colombia retoman relaciones diplomáticas rotas hace tres años,diplomáticas


In [54]:
stats_data.head()

,created_at,possibly_sensitive,id,retweet_count,reply_count,like_count,quote_count,referenced_tweets,newspaper,edit_history_tweet_ids,impression_count,year,week
index,,,,,,,,,,,,,
0,2022-08-28 23:57:24+00:00,False,1564039479391838209,0,0,6,1,None,elcomercio_peru,None,NaN,2022,34
2,2022-08-28 23:29:00+00:00,False,1564032331706470401,0,0,2,0,None,elcomercio_peru,None,NaN,2022,34
3,2022-08-28 23:14:11+00:00,False,1564028601053347843,6,7,18,1,None,elcomercio_peru,None,NaN,2022,34
5,2022-08-28 22:54:58+00:00,False,1564023766937731073,1,0,1,1,None,elcomercio_peru,None,NaN,2022,34
7,2022-08-28 22:30:25+00:00,False,1564017585561141248,3,0,8,0,None,elcomercio_peru,None,NaN,2022,34


In [55]:
data_dtm["year"] = data_dtm["created_at"].dt.isocalendar().year
data_dtm["week"] = data_dtm["created_at"].dt.isocalendar().week

In [56]:
top30_ids = pd.merge(data_dtm, top30_df,how="right", on=["newspaper", "year", "week", "word"])
top30_ids.dropna(subset=["id"], inplace=True)

In [57]:
top30_stats = pd.merge(top30_ids, stats_data, how="left", on=["id", "created_at", "newspaper", "year", "week"])

In [58]:
top30_stats.head()

,id,created_at,newspaper,corpus,word,year,week,count,hot_topics,possibly_sensitive,retweet_count,reply_count,like_count,quote_count,referenced_tweets,edit_history_tweet_ids,impression_count
0,1561458368761303040,2022-08-21 21:01:00+00:00,DiarioElPeruano,mayoría de fallecidos por accidentes de tránsito en lima tiene entre años revela la policía nacional del perú,nacional,2022,33,29.00,,False,2,0,9,0,None,None,NaN
1,1561378843180630016,2022-08-21 15:44:59+00:00,DiarioElPeruano,el titular del midagriperu andrés alencastre manifestó la necesidad de recuperar la institucionalidad de la autoridad nacional del agua anaperu generar mayores espacios de participación de las organizaciones de usuarios de riego,nacional,2022,33,29.00,,False,0,0,3,0,None,None,NaN
2,1561063176606826507,2022-08-20 18:50:39+00:00,DiarioElPeruano,el proceso de revisión construcción del currículo nacional permitirá contar con un documento diversificado flexible señaló el ministro de educación mineduperu rosendo serna,nacional,2022,33,29.00,,False,1,1,14,1,None,None,NaN
3,1561044528047370240,2022-08-20 17:36:32+00:00,DiarioElPeruano,especialistas del hospital nacional guillermo almenara de essaludperu reconstruyeron el esófago de una adolescente de años para que hoy pueda ingerir alimentos con normalidad luego de pasar años conectada una sonda gástrica,nacional,2022,33,29.00,,False,3,0,17,0,None,None,NaN
4,1561006996408733698,2022-08-20 15:07:24+00:00,DiarioElPeruano,el programa nacional de bienes incautados pronabi entregó en custodia con fines de asignación vehículos un inmueble seis embarcaciones instituciones públicas de las regiones piura cajamarca amazonas lambayeque,nacional,2022,33,29.00,,False,1,0,6,0,None,None,NaN


In [59]:
top30_stats_summary = top30_stats[["newspaper", "retweet_count", "reply_count", "like_count", "quote_count", "word", "year", "week"]].groupby(by=["newspaper", "year", "week", "word"]).agg(func=["count", "min", "mean", "std", "max", "sum"])

In [60]:
top30_stats_summary[("retweet_count", "ratio")] = top30_stats_summary[("retweet_count", "sum")] / top30_stats_summary[("retweet_count", "count")]
top30_stats_summary[("reply_count", "ratio")] = top30_stats_summary[("reply_count", "sum")] / top30_stats_summary[("reply_count", "count")]
top30_stats_summary[("like_count", "ratio")] = top30_stats_summary[("like_count", "sum")] / top30_stats_summary[("like_count", "count")]
top30_stats_summary[("quote_count", "ratio")] = top30_stats_summary[("quote_count", "sum")] / top30_stats_summary[("quote_count", "count")]

In [61]:
top30_stats_summary = top30_stats_summary.stack()
top30_stats_summary = top30_stats_summary.melt(var_name="metric", ignore_index=False)

In [62]:
top30_stats_summary = top30_stats_summary.reset_index()
top30_stats_summary.rename({"level_4": "stat"}, axis=1, inplace=True)

In [63]:
top30_stats_summary["year_week"] = top30_stats_summary["year"].astype("str") + "w" + top30_stats_summary["week"].astype("str")

In [64]:
top30_stats_summary.head()

,newspaper,year,week,word,stat,metric,value,year_week
0,DiarioElPeruano,2022,33,agosto,count,like_count,8.00,2022w33
1,DiarioElPeruano,2022,33,agosto,max,like_count,24.00,2022w33
2,DiarioElPeruano,2022,33,agosto,mean,like_count,8.50,2022w33
3,DiarioElPeruano,2022,33,agosto,min,like_count,2.00,2022w33
4,DiarioElPeruano,2022,33,agosto,ratio,like_count,8.50,2022w33


In [65]:
top30_stats_summary["hot_topics"] = top30_stats_summary["word"].map({
    "castillo": "castillo",
    "pedro": "castillo",
    "dina": "boluarte",
    "boluarte": "boluarte",
    "perú": "país",
    "país": "país",
    "congreso": "congreso",
    "covid": "covid",
    "protestas": "protestas",
    "manifestaciones": "protestas"
})
top30_stats_summary["hot_topics"].fillna("", inplace=True)

In [66]:
top30_stats_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58572 entries, 0 to 58571
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   newspaper   58572 non-null  object 
 1   year        58572 non-null  Int64  
 2   week        58572 non-null  Int64  
 3   word        58572 non-null  object 
 4   stat        58572 non-null  object 
 5   metric      58572 non-null  object 
 6   value       58572 non-null  float64
 7   year_week   58572 non-null  object 
 8   hot_topics  58572 non-null  object 
dtypes: Int64(2), float64(1), object(6)
memory usage: 4.1+ MB


In [67]:
top30_stats_summary.to_csv(f'{BASE_DIR}/reports/tables/top30_stats_summary-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.csv')

#### Building the app

In [5]:
top30_stats_summary = pd.read_csv(f'{BASE_DIR}/reports/tables/top30_stats_summary-{TIME_STAMPS[0]}-{TIME_STAMPS[-1]}.csv', index_col=0)
top30_stats_summary["hot_topics"].fillna("", inplace=True)

In [6]:
top30_app = JupyterDash(__name__)

In [7]:
top30_app.layout = html.Div(children=[
    html.H1("Top 30 words engagement stats per newspaper", style={"font-family": "Open Sans", "color": "#2a3f5f"}),
    html.Br(),
    html.Div([
        html.Div([
            html.Label("Stat", style={"font-family": "Open Sans", "color": "#2a3f5f"}),
                dcc.RadioItems(
                    top30_stats_summary["stat"].unique(),
                    "ratio",
                    id="stat",
                    inline=True,
                    style={"font-family": "Open Sans", "color": "#2a3f5f"})
            ]),
        html.Div([
            html.Label("Metric", style={"font-family": "Open Sans", "color": "#2a3f5f"}),
            dcc.RadioItems(
                top30_stats_summary["metric"].unique(),
                "like_count",
                id="metric",
                inline=True,
                style={"font-family": "Open Sans", "color": "#2a3f5f"}
                )]
            )],
        style={"display": "flex", "justify-content": "space-around"}),
    html.Br(),
    dcc.Graph(id="stats_graph")
], style={"height": "3600px"})

In [8]:
@top30_app.callback(
    Output("stats_graph", "figure"),
    Input("stat", "value"),
    Input("metric", "value")
    )
def update_figure(selected_stat, selected_metric):
    filtered_data = top30_stats_summary.loc[(top30_stats_summary["stat"] == selected_stat) & (top30_stats_summary["metric"] == selected_metric)]

    fig = px.bar(
        filtered_data,
        x="word",
        y="value",
        facet_row="newspaper",
        facet_col="year_week",
        color="hot_topics",
        color_discrete_sequence=gruvbox_colors,
        height=3200,
        width=3200
    )

    fig.for_each_annotation(lambda a: a.update(text=f"{a.text.split('=')[-1]}"))
    fig.update_xaxes(matches=None, showticklabels=True, categoryorder='total descending')
    fig.update_yaxes(matches=None, showticklabels=True)

    return fig

In [9]:
if __name__ == "__main__":
    top30_app.run_server(mode="inline")